In [9]:
import os
from pycocotools.coco import COCO
import tensorflow as tf
import numpy as np

annotation_path = '/content'
image_dir = '/content'
dataType = 'val2017'

annFile = os.path.join(annotation_path, 'annotations', f'instances_{dataType}.json')

coco = COCO(annFile)

target_classes = [1, 62]  # person and tv/monitor classes
cat_ids = coco.getCatIds(catNms=['person', 'tv'])

img_ids = coco.getImgIds(catIds=cat_ids)
images = coco.loadImgs(img_ids)

loading annotations into memory...
Done (t=3.86s)
creating index...
index created!


In [11]:
!pip install keras_cv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 27.9 MB/s eta 0:00:00


In [16]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model

# Load pre-trained EfficientDet model
base_model = tf.keras.applications.EfficientNetB0(input_shape=(None, None, 3), include_top=False)
base_model.trainable = False  # Freeze the base model

# Add custom layers for detection
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
output = layers.Dense(2, activation='sigmoid')(x)  # Two classes: person and monitor

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None, None, 3)  │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, None, None, 3)  │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization             │ (None, None, None, 3)  │              7 │ rescaling[0][0]        │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_1 (Rescaling)   │ (None, None, None, 3)  │              0 │ normalization[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, None, None, 3)  │              0 │ rescaling_1[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, None, None, 32) │            864 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, None, None, 32) │            128 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, None, None, 32) │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, None, None, 32) │            288 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, None, None, 32) │            128 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, None, None, 32) │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 32)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 32)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 8)        │            264 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_expand    

 Total params: 4,213,797 (16.07 MB)

 Trainable params: 164,226 (641.51 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [20]:
import cv2

def load_data():
    imgIds = coco.getImgIds()
    images = []
    bboxes = []
    classes = []

    for imgId in imgIds:
        img_info = coco.loadImgs(imgId)[0]
        image_path = os.path.join(image_dir, dataType, img_info['file_name'])
        image = cv2.imread(image_path)

        # Convert BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Load annotations for the image
        annIds = coco.getAnnIds(imgIds=imgId, iscrowd=None)
        annotations = coco.loadAnns(annIds)

        # Extract bounding boxes and class labels
        boxes = []
        class_labels = []
        for annotation in annotations:
            if annotation['category_id'] in [1, 43]:  # Assuming 1 is for 'person' and 43 is for 'tv'
                x, y, width, height = annotation['bbox']
                boxes.append([x, y, x + width, y + height])  # Convert to (x_min, y_min, x_max, y_max)
                class_labels.append(annotation['category_id'])

        if boxes:  # Only append images with annotations
            images.append(image)
            bboxes.append(boxes)
            classes.append(class_labels)

    return np.array(images), bboxes, classes

# Load the data
images, bboxes, classes = load_data()

error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
def preprocess_data(images, bboxes, classes):
    # Normalize images and prepare bounding boxes
    X = []
    y = []
    for img, box, cls in zip(images, bboxes, classes):
        img = cv2.resize(img, (224, 224)) / 255.0  # Resize and normalize
        X.append(img)

        # Convert bounding boxes and classes into the correct format
        y.append([box, cls])  # Custom processing might be needed

    return np.array(X), np.array(y)

# Preprocess the data
X_train, y_train = preprocess_data(images, bboxes, classes)

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
def predict(image):
    image_resized = cv2.resize(image, (224, 224)) / 255.0  # Resize and normalize
    predictions = model.predict(np.expand_dims(image_resized, axis=0))
    return predictions

# Load a test image
test_image_path = 'path/to/test_image.jpg'
test_image = cv2.imread(test_image_path)
predictions = predict(test_image)

# Display the predictions
print(predictions)
